# Test Notebook

In diesem Notebook werden alle Sachen geschrieben, die: 
- ausprobiert werden
- gerade nicht mehr eingesetzt werden
- Nur einmal für Datenverarbeitung benutzt werden, die nicht sein müsste

## Server DB richtig formatieren


In [1]:
import sqlite3
import pandas as pd
from db_connect import db_get_df, db_save_df

con = sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts_all.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()

OperationalError: unable to open database file

In [ ]:
len(df)

In [ ]:
print(df.dtypes)
print(df["filename"].drop_duplicates(ignore_index=True))

In [ ]:
df['filename'] = df['filename'].str.replace('/nfs/scratch/students/neumannvi84434/Podcast_Episoden/', '')

In [ ]:
with sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite") as con:
    df.to_sql("transcript_word_level_2237", con, index=False, if_exists='replace')

In [ ]:
db_save_df(df, "transcript_word_level_2237")

## Satzbildung durch Whisper Punkte

Wenn wir nun alle MP3 Datein transkribiert haben, ist für jedes einzelne vorkommende Wort ein Zeitstempel gespeichert.

Für das Projekt lohnt es sich allerdings größere Abschnitte zu erstellen, auf die später die Embeddings und die Suchen angewendet werden können, um mehr Kontext miteinzubeziehen.

In [5]:
from db_connect import db_get_df, db_save_df
from tqdm import tqdm
import pandas as pd

In [ ]:
df = db_get_df("transcript_word_level_2237")

In [ ]:
one_sentence = []
sentence_dict = [] 
filenames = df["filename"].drop_duplicates(ignore_index=True)

for filename in tqdm(filenames):
    word_entries = df[df['filename'] == filename]
    start = -1
    end = -1
    for index, word_entry in word_entries.iterrows(): 
        word = word_entry["word"]
        
        if not one_sentence: ##start
            start = word_entry["start"]
        one_sentence.append(word.strip())
        if "." in word:
            end = word_entry["end"]
            sentence = " ".join(one_sentence)
            sentence_dict.append({"filename": filename, "sentence": sentence, "start": start, "end":end})
            one_sentence = []



In [ ]:
df_sentences = pd.DataFrame(sentence_dict)

In [ ]:
len(df_sentences)

In [ ]:
db_save_df(df_sentences, "transcript_sentences_whisper")

## Setences ID hinzufügen

In [29]:
df =db_get_df("transcript_sentences")

In [3]:
df['segment_id'] = df.groupby('filename').cumcount()

In [30]:
df["u_id"] = [
        f'{row["filename"]}{row["sentence_id"]}' for _, row in df.iterrows()
    ]

In [31]:
db_save_df(df, "transcript_sentences")

## Kompositatrennung mit Pyphen (Silbentrennung)

In [ ]:
import pyphen
import os

deutsche_worte = pyphen.Pyphen(lang='de_DE')
einzelwoerter_set = set()

dateipfad = os.path.join(DATA_PATH, "vocabulary.txt")

with open(dateipfad, 'r', encoding='utf-8') as datei:
    for zeile in datei:
        woerter = zeile.strip().split()
        
        for wort in woerter:
            getrenntes_wort = deutsche_worte.inserted(wort)
            
            einzelwoerter = getrenntes_wort.split('-')
            einzelwoerter_set.update(einzelwoerter)

for wort in einzelwoerter_set:
    print(wort)


In [ ]:
print(len(einzelwoerter_set))
print(einzelwoerter_set[3000:3010])

In [ ]:
def save_list_to_file(lst, file_path):
    with open(file_path, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')

In [ ]:
einzelwoerter_set = sorted(einzelwoerter_set)
save_list_to_file(einzelwoerter_set, os.path.join(DATA_PATH, "vocabulary_pyphen.txt"))

## Vokabular compound splitten

In [ ]:
from german_compound_splitter import comp_split
from embedding_creation.embedding_creator_TF_IDF import is_number, compound_split_sentence
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()
AUDIO_SOURCE_PATH = os.getenv("AUDIO_SOURCE_PATH")
DATA_PATH = os.getenv("DATA_PATH")

In [ ]:
einzelwoerter_set = set()
dateipfad = os.path.join(DATA_PATH, "vocabulary.txt")
input_file = os.path.join(DATA_PATH, "german.dic")
ahocs = comp_split.read_dictionary_from_file(input_file)

In [ ]:
with open(dateipfad, 'r', encoding='utf-8') as datei:

    for zeile in tqdm(datei):
        word = zeile.strip()
        if is_number(word):
            continue
        try:
            dissection = comp_split.dissect(word, ahocs, make_singular=True)
        except:
            dissection = ""
        for split_word in dissection:
            einzelwoerter_set.add(split_word)

print(len(einzelwoerter_set))
# print(einzelwoerter_set)

In [ ]:
einzelwoerter_set = sorted(einzelwoerter_set)
save_list_to_file(einzelwoerter_set, os.path.join(DATA_PATH, "vocabulary_compound_split.txt"))

## debug db

In [2]:
from db_connect import db_get_df, db_save_df
from tqdm import tqdm
import pandas as pd

In [8]:
df = db_get_df("episodes_metadata")

In [3]:
df = db_get_df("transcript_sentences")

In [9]:
df =df[df["filename"] == "mars-mission-die-erforschung-des-roten-planeten.mp3"]

In [4]:
df = df["filename"].unique()

In [27]:
for filename in df:
    if "selbstwert" in filename:
        print(filename)

selbstwert-mut-zur-schwaeche.mp3


In [12]:
df["download_url"].values[0]

'https://media.neuland.br.de/file/1450661/c/feed/mars-mission-die-erforschung-des-roten-planeten.mp3'

In [6]:
len(df)

193

In [25]:
word = "schinkel"
indexes = []
for i, sentence in enumerate(df["sentence"]):
    if word in sentence:
        indexes.append(i)

for i in indexes:
    row = df.iloc[i]
    print(row["filename"], row["start"], row["sentence"])


e-t-a-hoffmann-ein-berufsbild.mp3 936.82 hoffmann erscheint legt sich samt foto murr schützend auf die poetischen erzeugnisse seines herren so als wolle er dessen nebentätigkeit verbergen was für ein kerlchen da vermag es nicht zu verwundern dass der dichte sogar eine todesanzeige in die zeitung setzt als murr in die ewigen jagdgründe entschwunden ist immer wieder wird die musik zum toast und eine junge unerreichbare sängerin mal wieder zur großen liebe johanna eunicke hoffmanns undine das libretto der oper stammt von lamotte fouquet das bühnenbild von schinkel die aufführung wird ein erfolg schließlich ist hoffmann inzwischen ein populärer autor dessen ausflüge in die musik man sich nicht entgehen lassen möchte 14 mal wird das stück in berlin aufgeführt und hätte wohl noch länger auf dem spielplan gestanden wäre nicht das schauspielhaus samt kostümen und dekorationen abgebrannt


## Test Langchain

In [28]:
import langchain
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
client = OpenAI(api_key=os.environ['OPENAI_KEY'])

In [5]:
llm_model = "gpt-3.5-turbo-0301"

In [23]:
def get_completion(prompt, model=llm_model):
        

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0, 
    )
    return response.choices[0].message.content

In [24]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

### Templates

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

### chaining

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [29]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [30]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [34]:
overall_simple_chain("protein powder")



> Entering new SimpleSequentialChain chain...
MuscleFuel
MuscleFuel is a sports nutrition company that produces high-quality supplements for building muscle, improving performance, and promoting overall health.

> Finished chain.


{'input': 'protein powder',
 'output': 'MuscleFuel is a sports nutrition company that produces high-quality supplements for building muscle, improving performance, and promoting overall health.'}

## test gpt

In [6]:
from segment_ranking.chatgpt_help import (
    gpt_segment_boundaries,
    gpt_filter_segments,
    gpt_get_keywords,
    gpt_order_segments,
)
from db_connect import db_get_df, db_append_df

In [2]:
df = db_get_df()

In [3]:
df_temp = df[df["filename"] == "spontanitaet-die-grosse-herausforderung.mp3"].reset_index(drop=True)

In [4]:
response = gpt_segment_boundaries(df_temp, "Marsmission")

In [5]:
response

,filename,start,end,sentence,segment_id
0,spontanitaet-die-grosse-herausforderung.mp3,1191.68,1260.84,Spontanität bedeutet immer auch eine Gradwande...,0


In [7]:
db_append_df(response, "best_fitting")

In [7]:
df_temp.loc[0:5, "sentence"]

0                                        Radio Wissen.
1                          Die ganze Welt des Wissens.
2    Ein Podcast von Bayern 2. Spontanität erleben ...
3    Dann haben wir vielleicht eine Ahnung davon, w...
4    Der deutsch -amerikanische Psychoanalytiker un...
5                         Die Furcht vor der Freiheit.
Name: sentence, dtype: object

In [10]:
response

{'Begründung': 'Der Textausschnitt diskutiert die Bedeutung und die Grenzen von Spontanität, was relevant sein könnte, wenn es um die Entscheidungsfindung und die Flexibilität bei einer Marsmission geht. Außerdem wird auf die Balance zwischen rationaler Planung und rein emotionalen Entscheidungen eingegangen, was bei der Planung einer Marsmission berücksichtigt werden sollte.',
 'Start': 164,
 'Ende': 172}

In [11]:
df_temp.iloc[82:89]

,filename,sentence,start,end,sentence_lemmatized,sentence_compound_split,sentence_id,segment_id
3673,spontanitaet-die-grosse-herausforderung.mp3,Stefan Grünewald erläutert diesen Mentalitätsu...,575.20,578.18,Stefan Grünewald erläutert diesen Mentalitätsu...,Stefan Grünewald diesen Mentalitätsunterschied,82,82
3674,spontanitaet-die-grosse-herausforderung.mp3,"Zum Deutschen naturell gehört, dass wir keine ...",578.56,583.80,"Zum Deutschen naturell gehört, dass wir keine ...",Zum Deut Ch Naturell Gehör wir keine klare Ide...,83,83
3675,spontanitaet-die-grosse-herausforderung.mp3,"Also die Franzosen, die haben so dieses Bild, ...",584.50,601.24,"Also die Franzosen, die haben so dieses Bild, ...",Also die Franzose die so dieses Bild wie Gott ...,84,84
3676,spontanitaet-die-grosse-herausforderung.mp3,"Das führt natürlich dazu, dass man ganz anders...",601.90,608.20,"Das führt natürlich dazu, dass man ganz anders...","Das führt DaZ man ganz anders gewillt ist, Ver...",85,85
3677,spontanitaet-die-grosse-herausforderung.mp3,"Das ist wie so eine sinnliche Expedition, die ...",608.52,614.28,"Das ist wie so eine sinnliche Expedition, die ...",Das ist wie so eine Sinn Expedit Ion die Dur d...,86,86
3678,spontanitaet-die-grosse-herausforderung.mp3,"Die Deutschen haben nicht so ein Bild, sie sin...",614.64,617.48,"Die Deutschen haben nicht so ein Bild, sie sin...",Die Deut Ch nicht so ein Bild sie sind ewig,87,87
3679,spontanitaet-die-grosse-herausforderung.mp3,"Das macht sie einerseits unruhig, das führt si...",617.48,630.76,"Das macht sie einerseits unruhig, das führt si...",Das Mac sie einerseits Unruh das führt sie and...,88,88


In [39]:
df["filename"][3626]

'spontanitaet-die-grosse-herausforderung.mp3'

In [13]:
df.iloc[0:0]

,filename,sentence,start,end,sentence_lemmatized,sentence_compound_split,sentence_id,segment_id


In [5]:
gpt_get_keywords(df_temp)

['Spontaneität und Werbung',
 'Spontaneität und Kaufverhalten',
 'Spontanität und Planung',
 'Spontaneität und kulturelle Unterschiede',
 'Spontaneität und kreative Prozesse']

In [ ]:
gpt_filter_segments(df_temp)

## Test PDF Reading

In [11]:
from audio_downloader.episodes_downloader import download_and_save_mp3_in_dir
import sys
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())
DATA_PATH = os.getenv("DATA_PATH")

In [14]:
path = os.path.join(DATA_PATH, "pdfs")
url = "https://www.br.de/radio/bayern2/service/manuskripte/radiowissen/manuskript-radiowissen-treue-wert-der-loyalitaet-100~attachment.pdf?version=ea7d8"
name = "manuskript-radiowissen-treue-wert-der-loyalitaet.pdf"
download_and_save_mp3_in_dir(url, path, name)

Downloaded https://www.br.de/radio/bayern2/service/manuskripte/radiowissen/manuskript-radiowissen-treue-wert-der-loyalitaet-100~attachment.pdf?version=ea7d8


In [18]:
from pypdf import PdfReader

reader = PdfReader(os.path.join(path, name))
page = reader.pages[1]
print(page.extract_text())

2 
 
Dieses Manuskript wird ohne Endkorrektur versandt und darf nur zum privaten Gebrauch verwendet werden.  
Jede andere Verwendung oder Veröffentlichung ist nur in Absprache mit dem Bayerischen Rundfunk möglich!   
Es gilt das gesprochene Wort.  
© Bayerischer Rundfunk 20 24 
Bayern 2 -Hörerservice  
Bayerischer Rundfunk, 80300 München; kostenlose Service -Nr.: 0800 -5900 222  
 service@bayern2.de ; www.bayern2.de   
 
Podcast:  
Treue in Beziehungen … das klingt in den Ohren mancher Menschen beinahe 
antiquiert. Ist der moderne Mensch doch frei, selbstbestimmt und unabhängig. 
Und wieso sollte er d as dann nicht auch in seinen Beziehungen sein?  
Natürlich waren sich Menschen in allen Zeiten zeitweise oder auch 
grundsätzlich untreu. Aber nur weil das so ist, muss das nicht heißen, dass die 
Treue wertlos geworden ist.  
 
 
Musik: Z9423981010 Parker & Hachi  walk to the station 0‘58  
 
Sprecherin:    
Es gibt diese Geschichte von Hachiko (ausgesprochen: Hadschiko ), einem 
japanis

install 

## Transkripte von Fraunhofer

In [7]:
from audio_downloader.episodes_downloader import get_graphql
from db_connect import db_get_df, db_save_df
import pandas as pd

In [4]:
query = """{
  programSet(id: 5945518) {
    title
    items(
      orderBy: PUBLISH_DATE_DESC
      filter: {
        isPublished: {
          equalTo: true
        }
      }
    ) {
      nodes {
        transcript {
          data
        }
        audios {
          downloadUrl
        }
      }
    }
  }
}
"""

In [5]:
data = get_graphql(query)

In [6]:
transcripts = []
audio_urls = []
for audio_data in data["data"]["programSet"]["items"]["nodes"]:
        transcripts.append(audio_data["transcript"])
        audio_urls.append(audio_data["audios"][0]["downloadUrl"])

In [15]:
audio_data["transcript"]["data"]

[{'end': '00:00:08.050',
  'text': 'Bayern zweiRadio präsentiert radioWissen immer werktags nach den neun Uhr Nachrichten Bayerns',
  'start': '00:00:00.000',
  'speaker': {'id': '1', 'name': 'unknown'}},
 {'end': '00:00:43.690',
  'text': 'In Europa nicht glücken selbst für die von die Schweizer gebe ich keine fünf Kreuzer. Anders duzt sich Österreich machen da gehen, umkehrt die Sache. Zwar ist die Aufgabe kleine Träne soll ein Zirkel Völkerschaften am ein Mittelpunkt pafften unsere Stellung, wo es schwierig und viele haben, schon gierig auf unsere Auflösung,',
  'start': '00:00:12.440',
  'speaker': {'id': '2', 'name': 'unknown'}},
 {'end': '00:01:24.580',
  'text': 'So heißt es in Johann Nestroy Stück Freiheit in Krawinkel aus dem Jahr 848. Schon damals war der Habsburger Vielvölkerstaat bedroht. Nestroy sah ein fragiles Gleichgewicht und den Zirkel Völkerschaften an einem Mittelpunkt haften. Zu jener Zeit waren die zentrifugalen Kräfte, die vom Mittelpunkt weg in alle Richtungen s

In [8]:
df = pd.DataFrame({"transcript": transcripts, "download_url":audio_urls})

In [16]:
len(df)

2253

In [20]:
print(len(df[df["transcript"].isna()]))
print(df[df["transcript"].isna()])

84
     transcript                                       download_url
0          None  https://media.neuland.br.de/file/2090876/c/fee...
3          None  https://media.neuland.br.de/file/2090855/c/fee...
4          None  https://media.neuland.br.de/file/2090034/c/fee...
7          None  https://media.neuland.br.de/file/2089948/c/fee...
8          None  https://media.neuland.br.de/file/1773273/c/fee...
...         ...                                                ...
162        None  https://media.neuland.br.de/file/2088143/c/fee...
180        None  https://media.neuland.br.de/file/2087498/c/fee...
188        None  https://media.neuland.br.de/file/2087347/c/fee...
276        None  https://media.neuland.br.de/file/2089215/c/fee...
2070       None  https://media.neuland.br.de/file/2088779/c/fee...

[84 rows x 2 columns]
